# Car Data Transformation Pipeline
## Transform raw JSON data to CSV datasets for RecSys & Chatbot

This notebook processes raw car data from JSON files and creates structured CSV datasets for:
- **Recommendation System**: Vehicle features, pricing, ratings
- **Chatbot**: Vehicle specifications, seller info, reviews

## 1. Import Libraries and Setup

In [ ]:
import json
import pandas as pd
import os
from pathlib import Path
import glob
from datetime import datetime
import numpy as np

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

print("Libraries imported successfully!")
print(f"Current working directory: {os.getcwd()}")

## 1.5. Setup Google Cloud Storage Connection

### Google Colab Authentication (Run this cell on Colab only)

If running on Google Colab, uncomment and run the authentication code below:

In [ ]:
# Uncomment these lines if running on Google Colab
# from google.colab import auth
# auth.authenticate_user()
# print("✓ Authenticated with Google Cloud")

# Alternative: Use gcloud commands (uncomment if needed)
# !gcloud auth application-default login
# !gcloud auth application-default set-quota-project car-recsys-consultant-chatbot

In [ ]:
# Install required package if not already installed
try:
    from google.cloud import storage
    print("✓ google-cloud-storage already installed")
except ImportError:
    print("Installing google-cloud-storage...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'google-cloud-storage', '-q'])
    from google.cloud import storage
    print("✓ google-cloud-storage installed successfully")

# Configuration
PROJECT_ID = "car-recsys-consultant-chatbot"  # Your GCP project ID
BUCKET_NAME = "car_recsys_consultant_chatbot"
BASE_PATH = "Car Recsys Consultant Chatbot"

print(f"\nProject ID: {PROJECT_ID}")
print(f"Bucket: {BUCKET_NAME}")
print(f"Base Path: {BASE_PATH}")

## 2. Load Raw Data from Google Cloud Storage

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from functools import partial

def download_and_parse_blob(blob, bucket_name):
    """Download and parse a single blob (for parallel processing)"""
    try:
        json_content = blob.download_as_text(encoding='utf-8')
        json_data = json.loads(json_content)
        json_data['_source_file'] = blob.name
        json_data['_gcs_path'] = f"gs://{bucket_name}/{blob.name}"
        return json_data, None
    except Exception as e:
        return None, (blob.name, str(e))

def load_json_files_from_gcs(project_id, bucket_name, base_path, max_workers=20):
    """
    Load all JSON files from Google Cloud Storage bucket with parallel processing
    
    Args:
        project_id: GCP project ID
        bucket_name: GCS bucket name
        base_path: Base path prefix in bucket
        max_workers: Number of parallel download threads (default: 20)
    """
    print("="*80)
    print("LOADING DATA FROM GOOGLE CLOUD STORAGE (OPTIMIZED)")
    print("="*80)
    
    try:
        # Initialize GCS client with project ID
        client = storage.Client(project=project_id)
        bucket = client.bucket(bucket_name)
        
        print(f"✓ Connected to project: {project_id}")
        print(f"✓ Connected to bucket: {bucket_name}")
        print(f"✓ Base path: {base_path}/")
        print(f"✓ Parallel workers: {max_workers}")
        print("\nScanning for JSON files...")
        
        # List all JSON blobs
        all_blobs = list(bucket.list_blobs(prefix=f"{base_path}/"))
        json_blobs = [blob for blob in all_blobs if blob.name.endswith('.json')]
        
        total_files = len(json_blobs)
        print(f"Found {total_files} JSON files")
        
        if total_files == 0:
            print("\n⚠ No JSON files found in the specified path")
            return []
        
        print(f"\nDownloading and parsing files (using {max_workers} parallel threads)...")
        
        data = []
        errors = []
        
        # Parallel download and parse
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            # Submit all tasks
            download_func = partial(download_and_parse_blob, bucket_name=bucket_name)
            future_to_blob = {executor.submit(download_func, blob): blob for blob in json_blobs}
            
            # Process completed tasks with progress bar
            completed = 0
            for future in as_completed(future_to_blob):
                completed += 1
                json_data, error = future.result()
                
                if json_data:
                    data.append(json_data)
                else:
                    errors.append(error)
                
                # Progress indicator every 50 files
                if completed % 50 == 0 or completed == total_files:
                    print(f"  Progress: {completed}/{total_files} files ({completed*100//total_files}%)")
        
        print("\n" + "="*80)
        print(f"✓ Successfully loaded: {len(data)} JSON files")
        if errors:
            print(f"✗ Errors: {len(errors)} files")
            print("\nFirst 5 errors:")
            for filename, error in errors[:5]:
                print(f"  - {filename}: {error}")
        print("="*80)
        
        return data
        
    except Exception as e:
        print(f"\n✗ Error connecting to GCS: {e}")
        print("\nPlease ensure:")
        print("1. You are authenticated with GCS")
        print("   - On Colab: Run authentication cell first")
        print("   - Local: gcloud auth application-default login")
        print("2. You have permission to access the bucket")
        print("3. The project ID, bucket name and path are correct")
        print(f"\nCurrent config:")
        print(f"  Project: {project_id}")
        print(f"  Bucket: {bucket_name}")
        print(f"  Path: {base_path}/")
        return []

# Load data from GCS with parallel processing
print("Starting data load...")
import time
start_time = time.time()

raw_data = load_json_files_from_gcs(PROJECT_ID, BUCKET_NAME, BASE_PATH, max_workers=20)

elapsed_time = time.time() - start_time
print(f"\n⏱ Total loading time: {elapsed_time:.2f} seconds")
if raw_data:
    print(f"   Average: {elapsed_time/len(raw_data):.3f} seconds per file")

# Display sample structure
if raw_data:
    print("\n" + "="*80)
    print("SAMPLE DATA STRUCTURE")
    print("="*80)
    sample = raw_data[0]
    print(f"Keys: {list(sample.keys())}")
    print(f"New/Used: {sample.get('post', {}).get('new_used', 'Unknown')}")
    print(f"Source: {sample.get('_source_file', 'Unknown')}")
    print(f"GCS Path: {sample.get('_gcs_path', 'Unknown')}")
    
    # Show distribution by condition
    print("\n" + "="*80)
    print("DATA DISTRIBUTION")
    print("="*80)
    conditions = {}
    for item in raw_data:
        condition = item.get('post', {}).get('new_used', 'Unknown')
        # Handle None values
        if condition is None:
            condition = 'Unknown'
        conditions[condition] = conditions.get(condition, 0) + 1
    
    # Sort with proper handling
    for condition in sorted(conditions.keys()):
        count = conditions[condition]
        print(f"  {condition}: {count} vehicles")
else:
    print("\n⚠ No data loaded. Please check your GCS configuration.")

## 3. Dataset 1: Vehicles (Used Cars)

In [ ]:
def extract_used_vehicles(data):
    """Extract vehicle information for USED cars"""
    vehicles = []
    
    for item in data:
        post = item.get('post') or {}
        car = item.get('car') or {}
        
        # Only process USED cars
        new_used = post.get('new_used', '')
        if not new_used or str(new_used).lower() != 'used':
            continue
        
        basics = post.get('basics_des') or {}
        user_history = post.get('user_history_des') or {}
        ratings = car.get('ratings') or {}
        metadata = item.get('_metadata') or {}
        
        vehicle = {
            # Basic Info
            'vehicle_id': basics.get('VIN', ''),
            'stock_number': basics.get('Stock #', ''),
            'condition': post.get('new_used', ''),
            'title': post.get('title', ''),
            'brand': car.get('brand', ''),
            'car_model': car.get('car_model', ''),
            'car_name': car.get('car_name', ''),
            
            # Pricing & Mileage
            'price': post.get('price', None),
            'monthly_payment': post.get('monthly_payment', None),
            'mileage': post.get('mileage', None),
            'mileage_str': basics.get('Mileage', ''),
            
            # Specifications
            'exterior_color': basics.get('Exterior color', ''),
            'interior_color': basics.get('Interior color', ''),
            'drivetrain': basics.get('Drivetrain', ''),
            'mpg': basics.get('MPG', ''),
            'fuel_type': basics.get('Fuel type', ''),
            'transmission': basics.get('Transmission', ''),
            'engine': basics.get('Engine', ''),
            'vin': basics.get('VIN', ''),
            
            # Vehicle History
            'accidents_damage': user_history.get('Accidents or damage', ''),
            'one_owner': user_history.get('1-owner vehicle', ''),
            'personal_use_only': user_history.get('Personal use only', ''),
            
            # Warranty
            'warranty': post.get('warranty_des', ''),
            
            # Ratings (from car reviews)
            'car_rating': car.get('car_rating', None),
            'percentage_recommend': car.get('percentage_recommend', None),
            'comfort_rating': ratings.get('Comfort', None),
            'interior_rating': ratings.get('Interior', None),
            'performance_rating': ratings.get('Performance', None),
            'value_rating': ratings.get('Value', None),
            'exterior_rating': ratings.get('Exterior', None),
            'reliability_rating': ratings.get('Reliability', None),
            
            # Links
            'vehicle_url': metadata.get('url', ''),
            'car_review_link': car.get('review_link', ''),
            'car_link': car.get('car_link', ''),
            
            # Metadata
            'source_file': item.get('_source_file', ''),
            'total_images': len(post.get('image') or []),
            'has_ratings': metadata.get('has_ratings', False),
            'data_complete': metadata.get('is_complete', False),
        }
        
        vehicles.append(vehicle)
    
    df = pd.DataFrame(vehicles)
    print(f"Extracted {len(df)} USED vehicles")
    return df

# Extract used vehicles
df_used_vehicles = extract_used_vehicles(raw_data)
print("\nUsed Vehicles Dataset Shape:", df_used_vehicles.shape)
print("\nColumn Names:")
print(df_used_vehicles.columns.tolist())
print("\nSample Data:")
df_used_vehicles.head()

## 4. Dataset 2: Vehicles (New Cars)

In [ ]:
def extract_new_vehicles(data):
    """Extract vehicle information for NEW cars"""
    vehicles = []
    
    for item in data:
        post = item.get('post') or {}
        car = item.get('car') or {}
        
        # Only process NEW cars
        new_used = post.get('new_used', '')
        if not new_used or str(new_used).lower() != 'new':
            continue
        
        basics = post.get('basics_des') or {}
        user_history = post.get('user_history_des') or {}
        ratings = car.get('ratings') or {}
        metadata = item.get('_metadata') or {}
        
        vehicle = {
            # Basic Info
            'vehicle_id': basics.get('VIN', ''),
            'stock_number': basics.get('Stock #', ''),
            'condition': post.get('new_used', ''),
            'title': post.get('title', ''),
            'brand': car.get('brand', ''),
            'car_model': car.get('car_model', ''),
            'car_name': car.get('car_name', ''),
            
            # Pricing & Mileage
            'price': post.get('price', None),
            'monthly_payment': post.get('monthly_payment', None),
            'mileage': post.get('mileage', None),
            'mileage_str': basics.get('Mileage', ''),
            
            # Specifications
            'exterior_color': basics.get('Exterior color', ''),
            'interior_color': basics.get('Interior color', ''),
            'drivetrain': basics.get('Drivetrain', ''),
            'mpg': basics.get('MPG', ''),
            'fuel_type': basics.get('Fuel type', ''),
            'transmission': basics.get('Transmission', ''),
            'engine': basics.get('Engine', ''),
            'vin': basics.get('VIN', ''),
            
            # Vehicle History (usually null for new cars)
            'accidents_damage': user_history.get('Accidents or damage', ''),
            'one_owner': user_history.get('1-owner vehicle', ''),
            'personal_use_only': user_history.get('Personal use only', ''),
            
            # Warranty
            'warranty': post.get('warranty_des', ''),
            
            # Ratings (usually null for new cars without reviews)
            'car_rating': car.get('car_rating', None),
            'percentage_recommend': car.get('percentage_recommend', None),
            'comfort_rating': ratings.get('Comfort', None),
            'interior_rating': ratings.get('Interior', None),
            'performance_rating': ratings.get('Performance', None),
            'value_rating': ratings.get('Value', None),
            'exterior_rating': ratings.get('Exterior', None),
            'reliability_rating': ratings.get('Reliability', None),
            
            # Links
            'vehicle_url': metadata.get('url', ''),
            'car_review_link': car.get('review_link', ''),
            'car_link': car.get('car_link', ''),
            
            # Metadata
            'source_file': item.get('_source_file', ''),
            'total_images': len(post.get('image') or []),
            'has_ratings': metadata.get('has_ratings', False),
            'data_complete': metadata.get('is_complete', False),
        }
        
        vehicles.append(vehicle)
    
    df = pd.DataFrame(vehicles)
    print(f"Extracted {len(df)} NEW vehicles")
    return df

# Extract new vehicles
df_new_vehicles = extract_new_vehicles(raw_data)
print("\nNew Vehicles Dataset Shape:", df_new_vehicles.shape)
print("\nColumn Names:")
print(df_new_vehicles.columns.tolist())
print("\nSample Data:")
df_new_vehicles.head()

## 5. Dataset 3: Sellers Information

In [ ]:
def extract_sellers(data):
    """Extract seller/dealer information"""
    sellers = []
    seen_sellers = set()
    
    for item in data:
        seller = item.get('seller') or {}
        
        if not seller:
            continue
        
        seller_key = seller.get('seller_key', '')
        
        # Avoid duplicates
        if seller_key in seen_sellers:
            continue
        seen_sellers.add(seller_key)
        
        phone_info = seller.get('phone_info') or {}
        hours = seller.get('hours') or {}
        
        seller_data = {
            # Basic Info
            'seller_key': seller_key,
            'seller_name': seller.get('seller_name', ''),
            'seller_link': seller.get('seller_link', ''),
            
            # Contact Info
            'phone_new': phone_info.get('New', ''),
            'phone_used': phone_info.get('Used', ''),
            'phone_service': phone_info.get('Service', ''),
            'destination': seller.get('destination', ''),
            
            # Business Hours
            'sales_hours': hours.get('Sales hours', ''),
            'service_hours': hours.get('Service hours', ''),
            'hours_monday': hours.get('Monday', ''),
            'hours_tuesday': hours.get('Tuesday', ''),
            'hours_wednesday': hours.get('Wednesday', ''),
            'hours_thursday': hours.get('Thursday', ''),
            'hours_friday': hours.get('Friday', ''),
            'hours_saturday': hours.get('Saturday', ''),
            'hours_sunday': hours.get('Sunday', ''),
            
            # Ratings
            'seller_rating': seller.get('seller_rating', None),
            'seller_rating_count': seller.get('seller_rating_count', None),
            
            # Description
            'description': seller.get('description', ''),
            
            # Images
            'total_images': len(seller.get('images') or []),
            'images_json': json.dumps(seller.get('images') or []),
            
            # Metadata
            'source_file': item.get('_source_file', ''),
        }
        
        sellers.append(seller_data)
    
    df = pd.DataFrame(sellers)
    print(f"Extracted {len(df)} unique sellers")
    return df

# Extract sellers
df_sellers = extract_sellers(raw_data)
print("\nSellers Dataset Shape:", df_sellers.shape)
print("\nColumn Names:")
print(df_sellers.columns.tolist())
print("\nSample Data:")
df_sellers.head()

## 6. Dataset 4: Vehicle Features (Used & New)

In [ ]:
def extract_vehicle_features(data):
    """Extract vehicle features (flattened from feature_des)"""
    features = []
    
    for item in data:
        post = item.get('post') or {}
        basics = post.get('basics_des') or {}
        feature_des = post.get('feature_des') or {}
        
        vehicle_id = basics.get('VIN', '')
        condition = post.get('new_used', '')
        
        if not feature_des:
            continue
        
        # Flatten features by category
        for category, feature_list in feature_des.items():
            if not isinstance(feature_list, list):
                continue
                
            for feature_name in feature_list:
                features.append({
                    'vehicle_id': vehicle_id,
                    'condition': condition,
                    'title': post.get('title', ''),
                    'feature_category': category,
                    'feature_name': feature_name,
                    'source_file': item.get('_source_file', ''),
                })
    
    df = pd.DataFrame(features)
    print(f"Extracted {len(df)} vehicle features")
    return df

# Extract vehicle features
df_vehicle_features = extract_vehicle_features(raw_data)
print("\nVehicle Features Dataset Shape:", df_vehicle_features.shape)
print("\nColumn Names:")
print(df_vehicle_features.columns.tolist())

# Show feature categories distribution
print("\nFeature Categories:")
print(df_vehicle_features['feature_category'].value_counts())

print("\nSample Data:")
df_vehicle_features.head(10)

## 7. Dataset 5: Reviews & Ratings (Used Cars Only)

In [ ]:
def extract_reviews(data):
    """Extract reviews and ratings from car model reviews (mainly for used cars)"""
    reviews = []
    
    for item in data:
        post = item.get('post') or {}
        car = item.get('car') or {}
        basics = post.get('basics_des') or {}
        
        # Only process items with car reviews
        if not car or not car.get('reviews'):
            continue
        
        vehicle_id = basics.get('VIN', '')
        condition = post.get('new_used', '')
        car_model = car.get('car_model', '')
        car_name = car.get('car_name', '')
        
        for review in car.get('reviews', []):
            ratings_breakdown = review.get('ratings_breakdown') or {}
            
            review_data = {
                # Vehicle Info
                'vehicle_id': vehicle_id,
                'condition': condition,
                'car_model': car_model,
                'car_name': car_name,
                'title': post.get('title', ''),
                
                # Review Info
                'overall_rating': review.get('overall_rating', None),
                'review_time': review.get('time', ''),
                'user_name': review.get('user_name', ''),
                'user_location': review.get('from', ''),
                'review_text': review.get('review', ''),
                
                # Detailed Ratings
                'comfort_rating': ratings_breakdown.get('Comfort', None),
                'interior_rating': ratings_breakdown.get('Interior', None),
                'performance_rating': ratings_breakdown.get('Performance', None),
                'value_rating': ratings_breakdown.get('Value', None),
                'exterior_rating': ratings_breakdown.get('Exterior', None),
                'reliability_rating': ratings_breakdown.get('Reliability', None),
                
                # Metadata
                'source_file': item.get('_source_file', ''),
            }
            
            reviews.append(review_data)
    
    df = pd.DataFrame(reviews)
    print(f"Extracted {len(df)} reviews")
    return df

# Extract reviews
df_reviews = extract_reviews(raw_data)
print("\nReviews Dataset Shape:", df_reviews.shape)
print("\nColumn Names:")
print(df_reviews.columns.tolist())
print("\nRating Distribution:")
print(df_reviews['overall_rating'].value_counts().sort_index(ascending=False))
print("\nSample Data:")
df_reviews.head()

## 8. Dataset 6: Vehicle Images

In [ ]:
def extract_vehicle_images(data):
    """Extract vehicle images with metadata"""
    images = []
    
    for item in data:
        post = item.get('post') or {}
        basics = post.get('basics_des') or {}
        
        vehicle_id = basics.get('VIN', '')
        condition = post.get('new_used', '')
        image_list = post.get('image') or []
        
        if not image_list:
            continue
        
        for idx, image_url in enumerate(image_list):
            images.append({
                'vehicle_id': vehicle_id,
                'condition': condition,
                'title': post.get('title', ''),
                'image_order': idx + 1,
                'image_url': image_url,
                'total_images': len(image_list),
                'source_file': item.get('_source_file', ''),
            })
    
    df = pd.DataFrame(images)
    print(f"Extracted {len(df)} vehicle images")
    return df

# Extract vehicle images
df_vehicle_images = extract_vehicle_images(raw_data)
print("\nVehicle Images Dataset Shape:", df_vehicle_images.shape)
print("\nColumn Names:")
print(df_vehicle_images.columns.tolist())
print("\nImages per vehicle statistics:")
print(df_vehicle_images.groupby('vehicle_id')['image_order'].count().describe())
print("\nSample Data:")
df_vehicle_images.head()

## 9. Dataset 7: Seller-Vehicle Relationship

In [ ]:
def extract_seller_vehicle_relationship(data):
    """Extract relationship between sellers and vehicles"""
    relationships = []
    
    for item in data:
        post = item.get('post') or {}
        seller = item.get('seller') or {}
        basics = post.get('basics_des') or {}
        metadata = item.get('_metadata') or {}
        
        vehicle_id = basics.get('VIN', '')
        seller_key = seller.get('seller_key', '')
        
        if not vehicle_id or not seller_key:
            continue
        
        relationships.append({
            'vehicle_id': vehicle_id,
            'seller_key': seller_key,
            'condition': post.get('new_used', ''),
            'title': post.get('title', ''),
            'seller_name': seller.get('seller_name', ''),
            'price': post.get('price', None),
            'stock_number': basics.get('Stock #', ''),
            'vehicle_url': metadata.get('url', ''),
            'source_file': item.get('_source_file', ''),
        })
    
    df = pd.DataFrame(relationships)
    print(f"Extracted {len(df)} seller-vehicle relationships")
    return df

# Extract seller-vehicle relationships
df_seller_vehicles = extract_seller_vehicle_relationship(raw_data)
print("\nSeller-Vehicle Relationship Dataset Shape:", df_seller_vehicles.shape)
print("\nColumn Names:")
print(df_seller_vehicles.columns.tolist())
print("\nSample Data:")
df_seller_vehicles.head()

## 10. Data Quality Check & Statistics

In [ ]:
print("="*80)
print("DATA QUALITY SUMMARY")
print("="*80)

datasets = {
    'Used Vehicles': df_used_vehicles,
    'New Vehicles': df_new_vehicles,
    'Sellers': df_sellers,
    'Vehicle Features': df_vehicle_features,
    'Reviews & Ratings': df_reviews,
    'Vehicle Images': df_vehicle_images,
    'Seller-Vehicle Relationships': df_seller_vehicles,
}

for name, df in datasets.items():
    print(f"\n{name}:")
    print(f"  - Total Records: {len(df)}")
    print(f"  - Columns: {len(df.columns)}")
    print(f"  - Memory Usage: {df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")
    
    # Check for missing values
    missing = df.isnull().sum()
    missing_pct = (missing / len(df) * 100).round(2)
    cols_with_missing = missing[missing > 0]
    
    if len(cols_with_missing) > 0:
        print(f"  - Columns with missing values: {len(cols_with_missing)}")
        print(f"    Top 3 missing:")
        for col in cols_with_missing.head(3).index:
            print(f"      • {col}: {missing[col]} ({missing_pct[col]}%)")
    else:
        print(f"  - No missing values!")

print("\n" + "="*80)
print("CONDITION DISTRIBUTION")
print("="*80)
print(f"\nUsed Vehicles: {len(df_used_vehicles)}")
print(f"New Vehicles: {len(df_new_vehicles)}")
print(f"Total: {len(df_used_vehicles) + len(df_new_vehicles)}")

# Price statistics
print("\n" + "="*80)
print("PRICE STATISTICS")
print("="*80)

if len(df_used_vehicles) > 0:
    print("\nUsed Vehicles:")
    print(df_used_vehicles['price'].describe())

if len(df_new_vehicles) > 0:
    print("\nNew Vehicles:")
    print(df_new_vehicles['price'].describe())

## 11. Save All Datasets to CSV Files

In [ ]:
# Create output directory
output_dir = Path("datasets")
output_dir.mkdir(exist_ok=True)

print("="*80)
print("SAVING DATASETS TO CSV FILES")
print("="*80)

# Save all datasets
datasets_to_save = {
    'used_vehicles.csv': df_used_vehicles,
    'new_vehicles.csv': df_new_vehicles,
    'sellers.csv': df_sellers,
    'vehicle_features.csv': df_vehicle_features,
    'reviews_ratings.csv': df_reviews,
    'vehicle_images.csv': df_vehicle_images,
    'seller_vehicle_relationships.csv': df_seller_vehicles,
}

for filename, df in datasets_to_save.items():
    filepath = output_dir / filename
    df.to_csv(filepath, index=False, encoding='utf-8')
    file_size = filepath.stat().st_size / 1024  # KB
    print(f"✓ Saved: {filename}")
    print(f"  - Records: {len(df)}")
    print(f"  - File size: {file_size:.2f} KB")
    print()

print("="*80)
print("ALL DATASETS SAVED SUCCESSFULLY!")
print("="*80)
print(f"\nOutput directory: {output_dir.absolute()}")
print(f"Total files: {len(datasets_to_save)}")

# List all created files
print("\nCreated files:")
for file in sorted(output_dir.glob("*.csv")):
    print(f"  - {file.name}")

## 12. Dataset Documentation & Use Cases

### Dataset Overview

#### 1. **used_vehicles.csv** (USED CARS)
**Purpose**: Main dataset for used car recommendation system
- **Key Fields**: vehicle_id, price, mileage, brand, model, ratings, specifications
- **Use Cases**:
  - Content-based filtering (by specs, features, price range)
  - Collaborative filtering (based on ratings)
  - Price prediction models
  - Vehicle history analysis

#### 2. **new_vehicles.csv** (NEW CARS)
**Purpose**: Main dataset for new car recommendation system
- **Key Fields**: vehicle_id, price, brand, model, specifications
- **Use Cases**:
  - New car recommendations
  - Price comparison
  - Feature-based filtering
  - Brand analysis

#### 3. **sellers.csv**
**Purpose**: Dealer/seller information for chatbot context
- **Key Fields**: seller_key, seller_name, ratings, contact_info, location
- **Use Cases**:
  - Dealer recommendations
  - Contact information retrieval
  - Seller rating analysis
  - Location-based search

#### 4. **vehicle_features.csv**
**Purpose**: Detailed feature breakdown for filtering and recommendation
- **Key Fields**: vehicle_id, feature_category, feature_name
- **Use Cases**:
  - Feature-based filtering ("cars with backup camera")
  - Feature importance analysis
  - Cross-vehicle feature comparison
  - RecSys feature engineering

#### 5. **reviews_ratings.csv** (USED CARS ONLY)
**Purpose**: User reviews and sentiment for recommendation quality
- **Key Fields**: vehicle_id, overall_rating, review_text, detailed_ratings
- **Use Cases**:
  - Sentiment analysis for chatbot responses
  - Rating-based recommendations
  - Review summarization
  - User feedback analysis

#### 6. **vehicle_images.csv**
**Purpose**: Vehicle image URLs for visual presentation
- **Key Fields**: vehicle_id, image_url, image_order
- **Use Cases**:
  - Visual display in chatbot/web interface
  - Image-based search (future enhancement)
  - Gallery generation

#### 7. **seller_vehicle_relationships.csv**
**Purpose**: Link vehicles to sellers for inventory queries
- **Key Fields**: vehicle_id, seller_key, price, stock_number
- **Use Cases**:
  - Inventory management
  - Seller-specific recommendations
  - Price comparison across sellers
  - Stock availability

## 13. Quick Data Exploration Examples

In [ ]:
# Example 1: Top 10 most reviewed car models (Used)
if len(df_reviews) > 0:
    print("="*80)
    print("TOP 10 MOST REVIEWED CAR MODELS")
    print("="*80)
    top_reviewed = df_reviews.groupby('car_name').agg({
        'overall_rating': ['count', 'mean']
    }).round(2)
    top_reviewed.columns = ['review_count', 'avg_rating']
    top_reviewed = top_reviewed.sort_values('review_count', ascending=False).head(10)
    print(top_reviewed)

# Example 2: Price range by brand (Used)
if len(df_used_vehicles) > 0 and 'brand' in df_used_vehicles.columns:
    print("\n" + "="*80)
    print("PRICE STATISTICS BY BRAND (USED VEHICLES)")
    print("="*80)
    brand_prices = df_used_vehicles[df_used_vehicles['brand'] != ''].groupby('brand')['price'].agg([
        'count', 'min', 'mean', 'max'
    ]).round(2)
    brand_prices = brand_prices.sort_values('mean', ascending=False).head(10)
    print(brand_prices)

# Example 3: Most common vehicle features
if len(df_vehicle_features) > 0:
    print("\n" + "="*80)
    print("TOP 15 MOST COMMON VEHICLE FEATURES")
    print("="*80)
    feature_counts = df_vehicle_features.groupby(['feature_category', 'feature_name']).size().reset_index(name='count')
    feature_counts = feature_counts.sort_values('count', ascending=False).head(15)
    print(feature_counts.to_string(index=False))

# Example 4: Seller ratings
if len(df_sellers) > 0:
    print("\n" + "="*80)
    print("TOP RATED SELLERS (with 100+ reviews)")
    print("="*80)
    top_sellers = df_sellers[df_sellers['seller_rating_count'] >= 100].sort_values(
        'seller_rating', ascending=False
    )[['seller_name', 'seller_rating', 'seller_rating_count', 'destination']].head(10)
    print(top_sellers.to_string(index=False))

## 14. Summary & Next Steps

### ✅ Transformation Complete!

#### What We Created:
1. **7 Clean CSV Datasets** ready for RecSys & Chatbot
2. **Separated Used vs New** vehicles for targeted analysis
3. **Normalized data structure** across all tables
4. **Rich feature extraction** from nested JSON

#### Key Separations:
- **Used Cars**: Include reviews, ratings, vehicle history
- **New Cars**: Fresh inventory with minimal history data
- **Sellers**: Centralized dealer information
- **Features**: Granular feature data for filtering
- **Reviews**: Sentiment and rating data for quality scoring

#### Next Steps for RecSys:
1. **Content-Based Filtering**: Use vehicle_features.csv + specifications
2. **Collaborative Filtering**: Use reviews_ratings.csv for user preferences
3. **Hybrid Approach**: Combine both with price, brand, ratings
4. **Feature Engineering**: Create derived features (price_per_mile, feature_count, etc.)

#### Next Steps for Chatbot:
1. **Load all CSVs** into your knowledge base
2. **Index reviews** for sentiment-based responses
3. **Link sellers** to vehicles for dealer queries
4. **Use features** for natural language filtering
5. **Integrate images** for visual responses

#### Data Quality Notes:
- Check for missing VINs (vehicle_id)
- Some new cars may lack reviews/ratings
- Validate price ranges for outliers
- Consider deduplication if needed